<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Advanced-LLM-Fine-Tuning/blob/main/Deep-seek-R1-MedicalSFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning DeepSeek-R1-Distill-Llama-8B

## Objective:
Adapt `DeepSeek-R1-Distill-Llama-8B` for medical chain-of-thought reasoning.

## Key Components:
- **Model:** `unsloth/DeepSeek-R1-Distill-Llama-8B`

> Add blockquote


- **Dataset:** 500 samples from `medical-o1-reasoning-SFT`
- **Tools:**
  - `Unsloth` (2x faster training)
  - 4-bit quantization
  - LoRA adapters
- **Result:** 44-minute training resulting in concise medical reasoning with structured `<think>` outputs.

## Performance Improvement:

| **Metric**         | **Before Fine-Tuning** | **After Fine-Tuning** |
|--------------------|------------------------|-----------------------|
| **Response Length** | 450 words              | 150 words             |
| **Reasoning Style** | Verbose                | Focused               |
| **Answer Format**   | Bulleted               | Paragraph             |


### step-by-step  fine-tune DeepSeek-R1-Distill-Llama-8B on medical data

##  **1: Install All the Required Packages**

In [ ]:
%%capture
# The '%%capture' magic command in Jupyter notebooks suppresses output from subsequent cells.

!pip install kaggle
# Installs the 'kaggle' package using pip. Assumes pip is installed and configured.

!pip install unsloth
# Installs the 'unsloth' package using pip. Similar assumption as above.

!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
# Installs a specific version of the 'unsloth' package directly from its GitHub repository.
# '--force-reinstall': Forces reinstallation even if the package is already installed.
# '--no-cache-dir': Avoids caching the installation files.
# '--no-deps': Skips installing dependencies, useful if dependencies are already satisfied.
# 'git+https://github.com/unslothai/unsloth.git': GitHub repository URL from which to install the package.


## **2: Authentication in Google Colab**

In [ ]:
from huggingface_hub import login
# Imports the 'login' function from the 'huggingface_hub' package to authenticate with Hugging Face.

from google.colab import userdata
# Imports 'userdata' from Google Colab, which allows access to stored secrets or credentials.

# Retrieve the Hugging Face token from Colab secrets
hf_token = userdata.get('HF_TOKEN')
# Gets the Hugging Face authentication token stored in Google Colab's 'userdata' for secure access.

# Log in to Hugging Face
login(hf_token)
# Uses the retrieved token to authenticate the user with Hugging Face's hub.

In [ ]:
import wandb
# Imports the 'wandb' library, which is used for experiment tracking and logging in machine learning.

from google.colab import userdata
# Imports 'userdata' from Google Colab to access stored secrets or credentials.

# Retrieve the Weights & Biases (W&B) API token from Colab secrets
wb_token = userdata.get('wandb')
# Gets the stored W&B API key to authenticate with the W&B platform.

wandb.login(key=wb_token)
# Logs into W&B using the retrieved API key for tracking experiments.

# Initialize a new W&B run for experiment tracking
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset',  # Specifies the W&B project name
    job_type="training",  # Labels this run as a training job
    anonymous="allow"  # Allows anonymous logging if authentication isn't provided
)


## **3: Model Initialization**

In [ ]:
from unsloth import FastLanguageModel
# Imports the 'FastLanguageModel' class from the 'unsloth' library, which is optimized for efficient language model training and inference.

# Define model configuration parameters
max_seq_length = 2048  # Sets the maximum sequence length for the model.
dtype = None  # Specifies the data type for model computation (None means the default type will be used).
load_in_4bit = True  # Enables 4-bit quantization for reduced memory usage and faster inference.

# Load the pretrained model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # Specifies the model to be loaded from Hugging Face Hub.
    max_seq_length=max_seq_length,  # Uses the defined max sequence length.
    dtype=dtype,  # Uses the specified data type (None defaults to the model’s recommended type).
    load_in_4bit=load_in_4bit,  # Enables 4-bit quantization if True.
    token=hf_token,  # Uses the Hugging Face authentication token to access private models if necessary.
)

## **4.Inference to model Model**

In [ ]:
from unsloth import FastLanguageModel

# Define the prompt format for inference
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

# Define the medical question for inference
question = (
    "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing "
    "but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would "
    "cystometry most likely reveal about her residual volume and detrusor contractions?"
)

# Prepare the model for inference mode
FastLanguageModel.for_inference(model)

# Tokenize the input prompt and move it to the GPU for efficient processing
inputs = tokenizer(
    [prompt_style.format(question, "")],  # Format the prompt with the question
    return_tensors="pt",  # Return PyTorch tensors
    padding=True,  # Ensure proper padding for batch processing
    truncation=True  # Prevent overly long inputs from causing issues
).to("cuda")  # Move tensors to GPU

# Generate model output based on the input prompt
outputs = model.generate(
    input_ids=inputs.input_ids,  # Input token IDs
    attention_mask=inputs.attention_mask,  # Attention mask for proper token processing
    max_new_tokens=1200,  # Limit the response length to avoid excessive output
    use_cache=True,  # Enable caching for faster inference
)

# Decode the generated output into a human-readable format
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Extract and print the response portion after "### Response:"
print(response[0].split("### Response:")[1].strip())

## **5.Dataset Preparation for Fine-Tuning**

In [ ]:
# Define the prompt style for training data
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
"""

# Retrieve the End-of-Sequence (EOS) token from the tokenizer
EOS_TOKEN = tokenizer.eos_token  # Ensures each generated text ends properly

def formatting_prompts_func(examples):
    """
    Formats input data into the specified prompt style for training.

    Args:
        examples (dict): A dictionary containing "Question", "Complex_CoT", and "Response".

    Returns:
        dict: A dictionary with formatted text prompts.
    """
    return {
        "text": [
            train_prompt_style.format(question, cot, response) + EOS_TOKEN
            for question, cot, response in zip(examples["Question"], examples["Complex_CoT"], examples["Response"])
        ]
    }

# Load the dataset from Hugging Face's Hub
from datasets import load_dataset

dataset = load_dataset(
    "FreedomIntelligence/medical-o1-reasoning-SFT",
    "en",
    split="train[0:500]",  # Selects the first 500 training examples
    trust_remote_code=True  # Allows loading datasets with external code dependencies
)

# Apply formatting function to dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Display the first formatted training example
dataset["text"][0]

## **6.Applying LoRA for Parameter-Efficient Fine-Tuning (PEFT) on Large Models**

In [ ]:
# Apply Parameter-Efficient Fine-Tuning (PEFT) to the model using LoRA (Low-Rank Adaptation)
# This allows fine-tuning large models with fewer resources by only updating a small subset of parameters
model = FastLanguageModel.get_peft_model(
    model,  # The pre-trained model to which LoRA will be applied
    r=16,  # Rank of the low-rank matrices used in LoRA. Higher values increase capacity but also computational cost.
           # Suggested values: 8, 16, 32, 64, 128. Choose based on your task and resources.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],  # List of model layers to apply LoRA to.
                                                           # These are typically attention and feed-forward layers.
    lora_alpha=16,  # Scaling factor for LoRA weights. Controls the magnitude of updates.
                    # A higher value increases the impact of LoRA updates.
    lora_dropout=0,  # Dropout rate for LoRA layers. Set to 0 for optimal performance.
                     # Dropout can help prevent overfitting but is not necessary here.
    bias="none",  # Whether to include bias terms in LoRA. "none" is optimized for efficiency.
    use_gradient_checkpointing="unsloth",  # Enables gradient checkpointing to save memory during training.
                                           # "unsloth" is optimized for very long sequences and reduces VRAM usage by 30%.
    random_state=3407,  # Random seed for reproducibility. Ensures consistent results across runs.
    use_rslora=False,  # Whether to use Rank-Stabilized LoRA (RS-LoRA). Set to False by default.
    loftq_config=None,  # Configuration for LoftQ (if applicable). Set to None as it is not used here.
)

## **7.Supervised Fine-Tuning with Optimized Training for Large Models**

In [ ]:
from trl import SFTTrainer
# Imports the 'SFTTrainer' class from the 'trl' (Transformer Reinforcement Learning) library,
# which is designed to facilitate supervised fine-tuning (SFT) of models.

from transformers import TrainingArguments
# Imports the 'TrainingArguments' class from the 'transformers' library, which contains configurations for model training.

from unsloth import is_bfloat16_supported
# Imports the 'is_bfloat16_supported' function from the 'unsloth' library, which checks if bfloat16 precision is supported.

# Initialize the SFTTrainer for fine-tuning the model
trainer = SFTTrainer(
    model=model,  # Specifies the pre-trained model to be fine-tuned.
    tokenizer=tokenizer,  # The tokenizer used for tokenizing inputs and outputs during training.
    train_dataset=dataset,  # The training dataset containing the formatted examples.
    dataset_text_field="text",  # Defines the column from the dataset that contains the text data (prompts).
    max_seq_length=max_seq_length,  # Maximum length of sequences to feed into the model during training.
    dataset_num_proc=2,  # Number of processes to use for data processing. Speed up data preparation.

    # Define training hyperparameters using the TrainingArguments class
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Batch size per device for training. Controls the number of samples processed in parallel.
        gradient_accumulation_steps=4,  # Number of steps to accumulate gradients before updating model weights.
                                        # Helps simulate larger batch sizes without needing additional GPU memory.
        # For full training runs, set num_train_epochs and warmup_ratio:
        warmup_steps=5,  # Number of steps for the learning rate warm-up.
        max_steps=60,  # Total number of training steps. Limits the training process to 60 steps.
        learning_rate=2e-4,  # Initial learning rate for the optimizer.
        fp16=not is_bfloat16_supported(),  # Use FP16 (half-precision) training if bfloat16 is not supported.
        bf16=is_bfloat16_supported(),  # Use BF16 (bfloat16) precision if supported by the hardware.
        logging_steps=10,  # Log the training progress every 10 steps.
        optim="adamw_8bit",  # Use the AdamW optimizer with 8-bit precision for efficiency.
        weight_decay=0.01,  # Weight decay applied to prevent overfitting.
        lr_scheduler_type="linear",  # Linear learning rate scheduler (gradual decay of learning rate).
        seed=3407,  # Random seed for reproducibility.
        output_dir="outputs",  # Directory to save the model outputs (checkpoints, logs).
    ),
)


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


### **8. Start Training**

In [ ]:
trainer_stats = trainer.train()
# Initiates the training process using the 'train' method of the SFTTrainer class.
# This method trains the model using the specified dataset, hyperparameters, and training configurations
# defined earlier in the 'trainer' object. It returns the training statistics, such as loss and accuracy,
# which are stored in the 'trainer_stats' variable for further analysis or logging.

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")


## 9. **Inference After Fine-Tuning**

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

# Define the format for the inference prompt
prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{question}

### Response:
{response}
"""

# Define the medical question for inference
question = (
    "A 45-year-old man with a history of alcohol use, who has been abstinent "
    "for the past 10 years, presents with sudden onset dysarthria, shuffling gait, "
    "and intention tremors. Given this clinical presentation and history, what is the most likely diagnosis?"
)

# Prepare the model for inference mode (prepares model for generating responses)
FastLanguageModel.for_inference(model)

# Tokenize the formatted prompt and move the tensors to GPU for efficient processing
inputs = tokenizer(
    [prompt_format.format(question=question, response="")],  # Format the prompt with the medical question
    return_tensors="pt",  # Return the tokenized inputs as PyTorch tensors
    padding=True,  # Ensure proper padding for batch processing
    truncation=True  # Prevent excessively long inputs from causing issues
).to("cuda")  # Move tensors to GPU for faster computation

# Initialize the TextStreamer to stream the output token-by-token
text_streamer = TextStreamer(tokenizer)

# Generate the model's output, streaming tokens in real-time as they are generated
_ = model.generate(
    **inputs,  # Provide the tokenized input text to the model for processing
    streamer=text_streamer,  # Enable token-by-token streaming to show real-time output
    max_new_tokens=1200  # Limit the response length to avoid excessively long outputs
)

## **10.Saving, loading finetuned models**
To save the final model as LoRA adapters

In [ ]:
# Save the trained model locally to a directory called "lora_model"
# This allows for later use of the fine-tuned model without retraining
model.save_pretrained("lora_model")

# Save the tokenizer associated with the model locally to the same "lora_model" directory
# The tokenizer is essential for processing inputs in the same way as during training
tokenizer.save_pretrained("lora_model")